# Basic Hands-On 2: Neural networks for regression - simple energy reconstruction with TA

We trained a simple model yesterday for the energy reconstruction from TA SD simulation. Today we will revise this task and train one of the proposed architectures we saw today to estimate the energy. No need for visualizing the data today.


### 1. Load data from ascii file 

We load the data as we did before and normalize the input features.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from IPython.display import display

# Set device (GPU if available, mps on Mac, else CPU)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")


In [2]:
class CosmicRayDataset(Dataset):
    def __init__(self, X_data, y_data):
        # Store the input features (X_data) and target values (y_data)
        self.X_data = X_data
        self.y_data = y_data

    def __len__(self):
        # Return the total number of samples in the dataset
        return len(self.X_data)

    def __getitem__(self, index):
        # Retrieve a single sample at the given index
        X_sample = self.X_data[index]
        y_sample = self.y_data[index]
        # return torch.tensor(X_sample, dtype=torch.float32), torch.tensor(y_sample, dtype=torch.float32)
        return X_sample, y_sample

In [3]:
# Load data again in case some variables have been overwritten
data_total_signal = np.loadtxt("../day1/total_signal_prot.txt", comments="#", dtype=np.float32)
data_arrival_times = np.loadtxt("../day1/arrival_times_prot.txt", comments="#", dtype=np.float32)

In [4]:
# Store data in a dictionary
data = dict()
data["energy"] = data_total_signal[:, 1]
data["shower_axis"] = data_total_signal[:, 2:5]
data["total_signal"] = data_total_signal[:, 5:].reshape(-1, 7, 7)
data["arrival_times"] = data_arrival_times[:, 5:].reshape(-1, 7, 7)

data["total_signal"] = (data["total_signal"] - data["total_signal"].min()) / (data["total_signal"].max() - data["total_signal"].min())
data["arrival_times"] = (data["arrival_times"] - data["arrival_times"].min()) / (data["arrival_times"].max() - data["arrival_times"].min())

In [32]:
data["total_signal"].shape

(51653, 7, 7)

In [5]:
# Reshape total_signal and arrival_time into 2D grids
total_signal = torch.tensor(data["total_signal"], dtype=torch.float32).to(device)  # [batch, 7, 7]
arrival_time = torch.tensor(data["arrival_times"], dtype=torch.float32).to(device)  # [batch, 7, 7]
energy = torch.tensor(data["energy"], dtype=torch.float32).to(device)

# Combine them into a 2-channel tensor
combined_features = torch.stack((total_signal, arrival_time), dim=1)  # Shape: [batch, 2, 7, 7]

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, energy, test_size=0.2, random_state=42)

# Reshape the data from (batch_size, 98) -> (batch_size, 2, 7, 7)
X_train_cnn = X_train.view(-1, 2, 7, 7)
X_test_cnn = X_test.view(-1, 2, 7, 7)

# Update DataLoader with new shapes
train_dataset = CosmicRayDataset(X_train_cnn, y_train)
test_dataset = CosmicRayDataset(X_test_cnn, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

### 2.  Train and evalulate the model

We are familiar with this step from the previous exercise and will simply take the same model for regression. We use the function from utils and modify it for regression.

In [33]:
def training_monitor(
    device,
    model,
    optimizer,
    criterion,
    num_epochs,
    trainloader,
    testloader,
    plot_interval=1,
):

    train_losses = []
    val_losses = []

    # Set up a single plot for loss
    fig, ax1 = plt.subplots(figsize=(10, 4))
    pbar = tqdm(total=num_epochs, leave=True)

    hdisplay_img = display(display_id=True)
    hdisplay_pbar = display(display_id=True)
    title = f"|{'Epoch':^20}|{'Train loss':^20}|{'Validation loss':^20}|"
    print(title)
    print("_" * len(title))

    for epoch in range(num_epochs):
        # Step 3: Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(trainloader)
        train_losses.append(train_loss)

        # Step 4: Validation phase
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in testloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(testloader)
        val_losses.append(avg_val_loss)

        # tqdm progress bar
        pbar.set_description(f"Epoch {epoch+1}/{num_epochs}")
        pbar.update()

        # Plotting updates every `plot_interval` epochs
        if (epoch + 1) % plot_interval == 0:
            # Clear the previous plot
            ax1.clear()

            # Plot training and validation loss
            ax1.plot(range(1, epoch + 2), train_losses, "b-", label="Train Loss")
            ax1.plot(range(1, epoch + 2), val_losses, "r-", label="Val Loss")
            ax1.set_xlabel("Epoch")
            ax1.set_ylabel("Loss")
            ax1.set_title("Loss")
            ax1.legend()
            ax1.grid(linestyle="--", linewidth=0.5, alpha=0.5)

            # Update the plot in the notebook
            hdisplay_img.update(fig)

        # Print losses at each epoch
        print(f"|{epoch+1:^20}|{train_loss:^20.4f}|{avg_val_loss:^20.4f}|")

    plt.close()

In [ ]:
# Evaluation function
def evaluate_cosmic_ray_model(model, X_test, y_test, criterion):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation for evaluation
        outputs = model(X_test)
        loss = criterion(outputs, y_test)
    print(f'Test Loss: {loss.item():.4f}')
    return outputs

# Exercise: 

Take one of the presented architectures and train a model for regression. Evaluate its performance with the training monitor function above and plot predicted energies vs. true energy. Discuss the performance and which steps to take for potential improvement.